## Libraries Required

In [30]:
from random import choice
import json
 
import requests
from bs4 import BeautifulSoup

from pprint import pprint
import urllib

from selenium import webdriver
import time

# Scrapping Code

#### To Do:

- Need to Write to CSV or Excel the file_paths
- Need to be able to grab the whole lot of images and not just limited to 12. By using selenium to continually scrape the page till the end

In [93]:
_user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'
]
 
class InstagramScraper:
        
    def using_selenium(self,url):
        browser = webdriver.Chrome('C://chromedriver.exe')
        browser.get(url)

        # Selenium script to scroll to the bottom, wait 3 seconds for the next batch of data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        match=False
        while(match==False):
            lastCount = lenOfPage
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount==lenOfPage:
                match=True
        source_data = browser.page_source
        bs_data = BeautifulSoup(source_data)
        return bs_data
    
    @staticmethod
    def extract_json_data(soup):
        body = soup.find('body')
        script_tag = body.find('script')
        raw_string = script_tag.text.strip().replace('window._sharedData =', '').replace(';', '')
        return json.loads(raw_string)
 
    def profile_page_metrics(self, profile_url):
        results = {}
        try:
            response = self.__request_url(profile_url)
            json_data = self.extract_json_data(response)
            metrics = json_data['entry_data']['ProfilePage'][0]['graphql']['user']
        except Exception as e:
            raise e
        else:
            for key, value in metrics.items():
                if key != 'edge_owner_to_timeline_media':
                    if value and isinstance(value, dict):
                        value = value['count']
                        results[key] = value
                    elif value:
                        results[key] = value
        return results

    
    def profile_page_recent_posts(self,profile_url):
        results = []
        try:
            response = self.using_selenium(profile_url)
            json_data = self.extract_json_data(response)
            metrics = json_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']["edges"]
        except Exception as e:
            raise e
        else:
            for node in metrics:
                node = node.get('node')
                if node and isinstance(node, dict):
                    results.append(node)
        return results

# Run Code here

Parse is just getting the names the images belongs to, for labels

In [64]:
url = 'https://www.instagram.com/lachlangorry/?hl=en'
parse = url.split('/')[3]

In [94]:
k = InstagramScraper()
results = k.profile_page_recent_posts(url)

for i in range(0,len(results)):
    image_src = results[i]['thumbnail_resources'][2]['src']
    file_names = parse+'_'+str(i)+'.jpg'
    file_path_to_save = './test_insta_scrapping_images/'
    urllib.request.urlretrieve(image_src,file_path_to_save+file_names)
    print(image_src,file_path_to_save+file_names)

https://scontent-syd2-1.cdninstagram.com/vp/189ea8fddd2be9bc2e1b535558264995/5CC68CFA/t51.2885-15/e35/c0.135.1080.1080/s320x320/47583721_518847321931179_8074330671801153944_n.jpg?_nc_ht=scontent-syd2-1.cdninstagram.com ./test_insta_scrapping_images/lachlangorry_0.jpg
https://scontent-syd2-1.cdninstagram.com/vp/334537fb56b438195cd4372acfbbd766/5CB70B59/t51.2885-15/e35/c0.135.1080.1080/s320x320/47692557_237704367129198_8921390728797885053_n.jpg?_nc_ht=scontent-syd2-1.cdninstagram.com ./test_insta_scrapping_images/lachlangorry_1.jpg
https://scontent-syd2-1.cdninstagram.com/vp/dfa088afd824cef2e862ea65580005d7/5CC7F106/t51.2885-15/e35/c135.0.809.809/s320x320/49678874_1867672110009443_7196953899411951838_n.jpg?_nc_ht=scontent-syd2-1.cdninstagram.com ./test_insta_scrapping_images/lachlangorry_2.jpg
https://scontent-syd2-1.cdninstagram.com/vp/58836b3749a6b68d2d6effabdf490643/5CBB4C35/t51.2885-15/e35/c0.135.1080.1080a/s320x320/49858045_313361219296917_8906536226002684063_n.jpg?_nc_ht=scontent-s